# Preprocessing

>Importing the right libraries to clean/examine the data

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
np.random.RandomState(42)

>Reading in the weather, training, spray, and testing dataset, and converting them into pandas dataframes.

In [2]:
train = pd.read_csv('../Data/train.csv')
weather = pd.read_csv('../Data/weather.csv')
spray = pd.read_csv('../Data/spray.csv')
test = pd.read_csv('../Data/test.csv')
train.name = 'train'
weather.name = 'weather'
spray.name = 'spray'
test.name = 'test'

### Cleaning the Weather Data

> According to the Data dictionary, the string 'M' and '-' were used to indicate missing data. By replacing them with null values, it will be easier to manage these missing values later. For CodeSum, a space represents no unusual weather, so we replace it for interpretability, For Precipitation total, the Data dictionary states that T stands for trace amount of participation, so it is reasonable to assume that this value could be half of the lowest value. This is done so that our data can be passed through a machine learning model.

In [3]:
weather.replace(to_replace='M',value=np.NAN,inplace = True)
weather.replace(to_replace='-',value=np.NAN,inplace = True)
weather['CodeSum'].replace(to_replace = ' ',value = 'No Weather',inplace = True)
weather['PrecipTotal'].replace(to_replace='  T',value=.005,inplace=True)

>Looking at the dates with missing precipitation, we can see that there are only two dates with that missing data. We can find these values online at https://www.wunderground.com/history/daily/us/il/chicago/KORD/date/2007-6-29. Both dates had no precipitation, so we can fill these missing values with 0.

In [4]:
weather[weather['PrecipTotal'].isnull()]

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,...,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
117,2,2007-06-28,73,61,67,NaN,56,61,0,2,...,No Weather,NaN,NaN,NaN,NaN,29.43,30.07,12.2,2,13.3
119,2,2007-06-29,71,56,64,NaN,56,60,1,0,...,No Weather,NaN,NaN,NaN,NaN,29.47,30.11,7.4,2,8.2


In [5]:
weather['PrecipTotal'].fillna(0.0,inplace=True)

>After doing outside research, I believe that these weather feature should have no direct impact in the probability of a trap having traces of West Nile Virus.

In [7]:
weather.drop(columns=['Water1',
                      'ResultSpeed',
                      'ResultDir',
                      'SeaLevel',
                      'AvgSpeed',
                      'SnowFall',
                      'Depth',
                      'Depart',
                      'StnPressure'],
             inplace=True)

>After looking at which features are missing data, we can see that most belong to station 2. These cells replace the missing data from station 2 with the data from station 1 for each day with missing data only.

In [8]:
print('There are',weather[weather['Station'] == 1].isnull().sum().sum(),'null values for station1')
print('There are',weather[weather['Station'] == 2].isnull().sum().sum(),'null values for station2')

There are 3 null values for station1
There are 2978 null values for station2


In [9]:
def check_nulls(dataframe):
    i = 0
    for col in dataframe:
        if dataframe[col].isnull().sum() > 0:
            i += 1
            print('The',col,'feature is missing',dataframe[col].isnull().sum(),'observations out of',len(dataframe))
    if i == 0:
        print('There are no missing values in the',dataframe.name,'dataframe')
check_nulls(weather)

The Tavg feature is missing 11 observations out of 2944
The WetBulb feature is missing 4 observations out of 2944
The Heat feature is missing 11 observations out of 2944
The Cool feature is missing 11 observations out of 2944
The Sunrise feature is missing 1472 observations out of 2944
The Sunset feature is missing 1472 observations out of 2944


In [10]:
station2_index = weather[weather['Station'] == 1].index+1
station1_index = weather[weather['Station'] == 1].index
for feature in ['Tavg','WetBulb','Sunrise','Sunset','CodeSum']:
    for row in station2_index:
        if weather.loc[row,feature] is np.NAN:
            weather.loc[row,feature] = weather.loc[row-1,feature]

>After applying our for loop, we can see that we are still missing 3 WetBulb values. Looking at these rows, we can see that they belong in station 1, so we apply the same concept that we applied above.

In [11]:
check_nulls(weather)

The WetBulb feature is missing 3 observations out of 2944
The Heat feature is missing 11 observations out of 2944
The Cool feature is missing 11 observations out of 2944


In [12]:
print('There are',weather[weather['Station'] == 1].isnull().sum().sum(),'null values for station1')
print('There are',weather[weather['Station'] == 2].isnull().sum().sum(),'null values for station2')

There are 3 null values for station1
There are 22 null values for station2


In [13]:
weather[weather['WetBulb'].isnull()]

,Station,Date,Tmax,Tmin,Tavg,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,CodeSum,PrecipTotal
848,1,2009-06-26,86,69,78,60,NaN,0,13,0418,1931,No Weather,0.00
2410,1,2013-08-10,81,64,73,57,NaN,0,8,0454,1900,No Weather,0.00
2412,1,2013-08-11,81,60,71,61,NaN,0,6,0455,1859,RA,0.01


In [14]:
weather.loc[station1_index,'WetBulb'] = weather.loc[station2_index,'WetBulb'].values

>According to the cell below, we can see that our dataframe is no longer missing any values.

In [15]:
check_nulls(weather)

The Heat feature is missing 11 observations out of 2944
The Cool feature is missing 11 observations out of 2944


> This is how our weather dataframe looks after cleaning

In [16]:
weather.head()

,Station,Date,Tmax,Tmin,Tavg,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,CodeSum,PrecipTotal
0,1,2007-05-01,83,50,67,51,57,0,2,0448,1849,No Weather,0.00
1,2,2007-05-01,84,52,68,51,57,0,3,0448,1849,No Weather,0.00
2,1,2007-05-02,59,42,51,42,47,14,0,0447,1850,BR,0.00
3,2,2007-05-02,60,43,52,42,47,13,0,0447,1850,BR HZ,0.00
4,1,2007-05-03,66,46,56,40,50,9,0,0446,1851,No Weather,0.00


### Cleaning Train/Test Data

>Because our training and testing dataset are the same (minus the output features of NumMosquitos and WnvPresent), we should clean them in the same way in order to make sure that they have the same shape. This is important for our machine learning model's prediction process.

>The cell below drops the features that have to do with location that we can not use while creating our machine learning models, like the Address, Street, etc.

In [17]:
for df in [test,train]:
    df.drop(columns=['Street','AddressNumberAndStreet','Address','AddressAccuracy'],inplace = True)

In [18]:
check_nulls(test)
check_nulls(train)

There are no missing values in the test dataframe
There are no missing values in the train dataframe


In [19]:
train.head()

,Date,Species,Block,Trap,Latitude,Longitude,NumMosquitos,WnvPresent
0,2007-05-29,CULEX PIPIENS/RESTUANS,41,T002,41.954690,-87.800991,1,0
1,2007-05-29,CULEX RESTUANS,41,T002,41.954690,-87.800991,1,0
2,2007-05-29,CULEX RESTUANS,62,T007,41.994991,-87.769279,1,0
3,2007-05-29,CULEX PIPIENS/RESTUANS,79,T015,41.974089,-87.824812,1,0
4,2007-05-29,CULEX RESTUANS,79,T015,41.974089,-87.824812,4,0


### Cleaning Spray Data

> Because there is no way of filling the missing values in our spray data, we would either have to remove the observations that have no Time values, or drop the Time feature. I believe that not knowing the time of the spray is better than not knowing the location of 500+ sprays.

In [20]:
check_nulls(spray)

The Time feature is missing 584 observations out of 14835


In [21]:
spray.drop(columns='Time',inplace=True)
spray.head()

,Date,Latitude,Longitude
0,2011-08-29,42.391623,-88.089163
1,2011-08-29,42.391348,-88.089163
2,2011-08-29,42.391022,-88.089157
3,2011-08-29,42.390637,-88.089158
4,2011-08-29,42.390410,-88.088858


### Saving the Clean Data Frames

In [22]:
spray.to_csv('../Data/clean_spray.csv')
test.to_csv('../Data/clean_test.csv')
weather.to_csv('../Data/clean_weather.csv')
train.to_csv('../Data/clean_train.csv')